<a href="https://colab.research.google.com/github/FabianWalocha/sentimentAnalysis/blob/master/CV_CNNLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##CNN-LSTM approach

In [0]:
%cd /content
!rm -rf sentimentAnalysis
!git clone https://github.com/FabianWalocha/sentimentAnalysis.git
%cd sentimentAnalysis/

/content
Cloning into 'sentimentAnalysis'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 103 (delta 63), reused 63 (delta 30), pack-reused 0
Receiving objects: 100% (103/103), 6.28 MiB | 13.39 MiB/s, done.
Resolving deltas: 100% (63/63), done.
/content/sentimentAnalysis


In [0]:
%tensorflow_version 1.x
from sklearn import metrics
from random import shuffle
from collections import defaultdict
from tqdm import tqdm_notebook as tqdm
from gensim.models import Word2Vec, Doc2Vec
from gensim.models.doc2vec import LabeledSentence
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC 
from keras.models import load_model
import matplotlib.pyplot as plt
import seaborn as sns
import  numpy as np
from matplotlib import pyplot as plt
import spacy
import os
import warnings
from CNNLSTM import score_CNN_LSTM as m1
from w2vLSTM import score_CNN_LSTM as m2
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [0]:
path_data = 'tweet_data_pre_processed.txt'

In [0]:
labeled_tweets = []
with open(path_data,'r', encoding='utf-8') as f:
    for tweet in f:
        t  = tweet.split(" ")
        labeled_tweets.append((t[0]," ".join(t[1:])))
shuffle(labeled_tweets)
tweets = [x for _,x in labeled_tweets]
labels = [y for y,_ in labeled_tweets]

In [0]:
# from gensim.models import Word2Vec, Doc2Vec
# from gensim.models.doc2vec import LabeledSentence
# vector_size = 200
# def train_model(data):
#     print('Training Doc2Vec model...')
#     epochs = 15
#     model = Doc2Vec(min_count=3, window=2, size=vector_size)
#     model.build_vocab(data)
#     model.train(data,total_examples=model.corpus_count,epochs=epochs)
#     return model

In [0]:
print('Generating cross-folds...')
split_ratio = (0.8,0.1,0.1) #Train,Dev,Test
cross_folds = int((1-split_ratio[0])**(-1))
print(f'Using {cross_folds} crossfolds...')
assert sum(split_ratio) == 1
cross_fold_items = []
print('Training classifiers...')
for i in range(cross_folds):
    size = len(tweets)
    k1 = i*int(size*split_ratio[1])
    k2 = k1+int(size*split_ratio[1])
    k3 = k2+int(size*split_ratio[2])

    trainD = tweets[k3:] + tweets[:k1]
    testD = tweets[k1:k2]
    devD = tweets[k2:k3]
    print('Generating training data...')
    training_data = trainD
    training_labels = np.hstack(label for label in labels[k3:] + labels[:k1])
    print('Generating test data...')
    test_data = testD
    test_labels = np.hstack(label for label in labels[k1:k2])
    print('Generating dev data...')
    dev_data = devD
    dev_labels = np.hstack(label for label in labels[k2:k3])
    cross_fold_items.append((training_data,training_labels,test_data,test_labels,dev_data,dev_labels))
    assert len(test_data) + len(training_data) + len(dev_data) == size

Generating cross-folds...
Using 5 crossfolds...
Training classifiers...
Generating training data...
Generating test data...
Generating dev data...
Generating training data...
Generating test data...
Generating dev data...
Generating training data...
Generating test data...
Generating dev data...
Generating training data...
Generating test data...
Generating dev data...
Generating training data...
Generating test data...
Generating dev data...


In [0]:
import numpy as np
print('Classifiers...')
lossT, accT, f1T, precT, recT = 0,0,0,0,0
res = np.zeros([5,5])
for i in range(cross_folds):
    print('#'*35)
    training_data,training_labels,test_data,test_labels,dev_data,dev_labels = cross_fold_items[i]
    loss, acc, f1, prec, rec = m1(training_data,training_labels,dev_data,dev_labels,test_data,test_labels, epochs=20, verbose=0)
    res[i,:] = [loss, acc, f1, prec, rec]

Classifiers...
###################################









Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




5033/5033 [==============================] - 1s 222us/step
###################################
5033/5033 [==============================] - 2s 314us/step
###################################
5033/5033 [==============================] - 2s 427us/step
###################################
5033/5033 [==============================] - 3s 509us/step
###################################
5033/5033 [==============================] - 3s 603us/step


In [0]:
import pandas as pd

res_df = pd.DataFrame(data=np.round(res,3),columns=['loss','accuracy','f1_score','precision','recall'])
res_df

,loss,accuracy,f1_score,precision,recall
0,0.440,0.615,0.615,0.615,0.615
1,0.414,0.625,0.625,0.625,0.625
2,0.423,0.621,0.621,0.621,0.621
3,0.432,0.624,0.624,0.624,0.624
4,0.449,0.588,0.588,0.588,0.588


In [0]:
print(res_df.mean())
print()
print(res_df.std())

loss         0.4316
accuracy     0.6146
f1_score     0.6146
precision    0.6146
recall       0.6146
dtype: float64

loss         0.013759
accuracy     0.015372
f1_score     0.015372
precision    0.015372
recall       0.015372
dtype: float64


In [0]:
from gensim.models import Word2Vec, Doc2Vec
from gensim.models.doc2vec import LabeledSentence
vector_size = 200
def train_model(data):
    print('Training Doc2Vec model...')
    epochs = 15
    model = Doc2Vec(min_count=3, window=2, size=vector_size)
    model.build_vocab(data)
    model.train(data,total_examples=model.corpus_count,epochs=epochs)
    return model

In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

print('Generating cross-folds...')
split_ratio = (0.8,0.1,0.1) #Train,Dev,Test
cross_folds = int((1-split_ratio[0])**(-1))
print(f'Using {cross_folds} crossfolds...')
assert sum(split_ratio) == 1
cross_fold_items = []
print('Training classifiers...')
for i in range(cross_folds):
    size = len(tweets)
    k1 = i*int(size*split_ratio[1])
    k2 = k1+int(size*split_ratio[1])
    k3 = k2+int(size*split_ratio[2])

    trainD = list(enumerate(tweets[k3:] + tweets[:k1]))
    testD = list(enumerate(tweets[k1:k2]))
    devD = list(enumerate(tweets[k2:k3]))
    print('Generating training data...')
    model_train = train_model([LabeledSentence(tweet,[str(id_)]) for id_,tweet in trainD])
    training_data = np.vstack(model_train[str(id_)] for id_,_ in trainD)
    training_labels = np.hstack(label for label in labels[k3:] + labels[:k1])
    print(training_labels[:5])
    print('Generating test data...')
    model_test =  [model_train.infer_vector(tweet) for id_,tweet in testD]      
    test_data = np.vstack(model_test[id_] for id_,_ in testD)
    test_labels = np.hstack(label for label in labels[k1:k2])
    print('Generating dev data...')
    model_dev =  [model_train.infer_vector(tweet) for id_,tweet in devD]    
    dev_data = np.vstack(model_dev[id_] for id_,_ in devD)
    dev_labels = np.hstack(label for label in labels[k2:k3])

    clf = LinearSVC().fit(training_data, training_labels)
    y_pred = clf.predict(test_data)
    print(classification_report(test_labels, y_pred))

    cross_fold_items.append((training_data,training_labels,test_data,test_labels,dev_data,dev_labels,model_train,model_test,model_dev))
    assert len(test_data) + len(training_data) + len(dev_data) == size

Generating cross-folds...
Using 5 crossfolds...
Training classifiers...
Generating training data...
Training Doc2Vec model...
['positive' 'neutral' 'neutral' 'negative' 'negative']
Generating test data...
Generating dev data...
              precision    recall  f1-score   support

    negative       0.31      0.02      0.04       751
     neutral       0.51      0.56      0.53      2284
    positive       0.46      0.58      0.51      1998

    accuracy                           0.48      5033
   macro avg       0.43      0.38      0.36      5033
weighted avg       0.46      0.48      0.45      5033

Generating training data...
Training Doc2Vec model...
['neutral' 'positive' 'neutral' 'positive' 'neutral']
Generating test data...
Generating dev data...
              precision    recall  f1-score   support

    negative       0.33      0.03      0.06       830
     neutral       0.50      0.57      0.53      2252
    positive       0.46      0.55      0.50      1951

    accuracy      